<a href="https://colab.research.google.com/github/biplobsd/swirl-colab/blob/main/swirl_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![swirl logo](https://swirl.today/wp-content/uploads/2023/07/swirl-company-logo-white-250x100-1-1.png)

Swirl is open-source, enterprise search and AI

This notebook created by `Biplob Kumar Sutradhar` - [biplobsd.me](https://biplobsd.github.io)

Read more - [https://biplobsd.me/blogs/view/run-swirl-open-source-search-engine-on-google-colab.md](https://biplobsd.me/blogs/view/run-swirl-open-source-search-engine-on-google-colab.md)

## Step 1 - Prerequisites Installation

In [ ]:
%%shell
pip install udocker
udocker --allow-root install
apt-get install redis-server
redis-server --daemonize yes

## Step 2 - Port forward 8000

Do not click after getting the link. `Step 3` is required and running successfully then click the forward link.

In [ ]:
#@markdown <center><h3>Port forwarding</h3></center>
import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output


#####################################
USE_FREE_TOKEN = True
TOKEN = ""
PORT_LIST = "8000" #@param {type:"string"}

#@markdown <font size=2px>Input PORT which you can want to forward. Separate two PORT between by comma. <br>Ex `80, 8080, 4040`</font>
REGION = "US" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl,
    textAn
)

loadingAn()

os.makedirs('tools/', exist_ok=True)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
m = []
splitPortList = PORT_LIST.split(',')
for p in splitPortList:
  p = int(p)
  m.append([f"s{p}", p, 'http'])


PORT_FORWARD = "localhost" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, m, REGION.lower(),
    [f"{HOME}/.ngrok2/randomPortOpen.yml", 45535]
)

for l in m:
  displayUrl(Server.start(l[0], displayB=False, v=False),
             pNamU=f"{l[0][1:]} -> ", cls=False)

## Step 3 - start swirl using udocker

Default username: `admin` and password: `password`

___Please wait for 1m:30s for booting the image.___

In [15]:
%%shell
udocker --allow-root run \
  -p 8000:8000 \
  -p 6379:6379 \
  -e ALLOWED_HOSTS="*" \
  -e CELERY_BROKER_URL=redis://127.0.0.1:6379/0 \
  -e CELERY_RESULT_BACKEND=redis://127.0.0.1:6379/0 \
  swirlai/swirl-search:latest \
  sh -c 'rm -fr ./.swirl &&
         python swirl.py setup &&
         /usr/bin/jq ".default" ./config-swirl-demo.db.json | sed -e "s/<msal-app-id>/$MSAL_APP_ID/" \
                                                              -e "s/<msal-tenant-id>/$MSAL_TENANT_ID/" \
                                                              -e "s/<msal-port>/$MSAL_CB_PORT/" \
                                                              -e "s/<msal-host>/$MSAL_HOST/" > static/api/config/default &&
         python swirl.py start celery-worker celery-beats &&
         daphne -b 0.0.0.0 -p 8000 swirl_server.asgi:application'

 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING ae8ce180-364e-350e-80b5-276c96f30ff3                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: sh
__S_W_I_R_L__2_._6______________________________________________________________

Setting Up SWIRL:
Checking Migrations:
No changes detected


Migrating:

Operations to perform:
  Apply all migrations: admin, auth, authtoken, contenttypes, django_celery_beat, sessions, swirl
Running migrations:
  No migrations to apply.




215 static files copied to '/app/static'.

Ok
Command successful!
__S_W_I_R_L__2_._6______________________________________________________________

ParseResult(scheme='redis', netloc='127.0.0.1:6379', path='/0', params='', query='', fragment='') checke

CalledProcessError: ignored